In [1]:
!pip install langchain_community
!pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 966.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52

In [2]:
import os
os.environ['OPENAI_API_KEY'] = 'your api key'

In [3]:
import inspect
import re

from langchain.prompts import PromptTemplate
from langchain.chains import  LLMMathChain, TransformChain, SequentialChain
from langchain_openai import OpenAI
from langchain_core.runnables import RunnableSequence
from langchain_community.callbacks import get_openai_callback


In [4]:
llm = OpenAI(temperature=0)

In [5]:
llm_math = LLMMathChain.from_llm(OpenAI(), verbose=True)
result = llm_math.invoke("What is area of a circle with radius 3.9?")  # Run the chain with the question



> Entering new LLMMathChain chain...
What is area of a circle with radius 3.9?```text
pi * (3.9)**2
```
...numexpr.evaluate("pi * (3.9)**2")...

Answer: 47.78362426110075
> Finished chain.


In [6]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.invoke(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [7]:
llm_math = LLMMathChain.from_llm(OpenAI(), verbose=True)
count_tokens(llm_math, "What is 13 raised to the .3432 power?")



> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?```text
13 ** .3432
```
...numexpr.evaluate("13 ** .3432")...

Answer: 2.4116004626599237
> Finished chain.
Spent a total of 226 tokens


{'question': 'What is 13 raised to the .3432 power?',
 'answer': 'Answer: 2.4116004626599237'}

In [8]:
print(llm_math.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



In [9]:
print(inspect.getsource(llm_math._call))

    def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        _run_manager.on_text(inputs[self.input_key])
        llm_output = self.llm_chain.predict(
            question=inputs[self.input_key],
            stop=["```output"],
            callbacks=_run_manager.get_child(),
        )
        return self._process_llm_result(llm_output, _run_manager)



In [10]:
# we set the prompt to only have the question we ask
prompt = PromptTemplate(input_variables=['question'], template='{question}')
llm_chain = LLMMathChain.from_llm(OpenAI())
# we ask the llm for the answer with no context

count_tokens(llm_chain, "What is 13 raised to the .3432 power?")

Spent a total of 226 tokens


{'question': 'What is 13 raised to the .3432 power?',
 'answer': 'Answer: 2.4116004626599237'}

In [11]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]

    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

In [12]:
clean_extra_spaces_chain = TransformChain(input_variables=["text"], output_variables=["output_text"], transform=transform_func)

In [13]:
clean_extra_spaces_chain.invoke('A random text  with   some irregular spacing.\n\n\n     Another one   here as well.')

{'text': 'A random text  with   some irregular spacing.\n\n\n     Another one   here as well.',
 'output_text': 'A random text with some irregular spacing.\n Another one here as well.'}

In [14]:
template = """Paraphrase this text:

{output_text}

In the style of a {style}.

Paraphrase: """
prompt = PromptTemplate(input_variables=["style", "output_text"], template=template)

In [15]:
style_paraphrase_chain = RunnableSequence(prompt | llm)

In [16]:
sequential_chain = RunnableSequence(
    clean_extra_spaces_chain, style_paraphrase_chain)





In [17]:
input_text = """
deep learning is at the core of large                                  language models,
                  enabling them to understand and generate human language.                These models have achieved remarkable
                   success in various language-related tasks,
                    revolutionizing
the field of natural
                       language processing.
                       One of the key advantages of LLMs is their ability to perform transfer learning. They can be pre-trained on a large dataset and then fine-tuned for specific tasks.
        This
          reduces the need for large task-specific                      datasets and accelerates the development of natural                       language processing
          applications.
"""

In [18]:
count_tokens(sequential_chain, {'text': input_text, 'style': 'a 90s rapper'})

Spent a total of 226 tokens


"Yo, deep learning be the heart of them big language models, makin' 'em able to comprehend and spit out human words. These models be killin' it in all sorts of language tasks, straight up changin' the game in natural language processing. One dope thing about LLMs is they can do transfer learnin', gettin' pre-trained on a fat dataset and then tweaked for specific jobs. This means less need for huge task-specific data and faster development of language processing apps. Word."

In [19]:
count_tokens(sequential_chain, {'text': input_text, 'style': 'formal'})

Spent a total of 213 tokens


'\nDeep learning serves as the foundation for extensive language models, empowering them to comprehend and produce human language. These models have demonstrated exceptional accomplishments in diverse language-oriented assignments, fundamentally transforming the realm of natural language processing. A significant benefit of LLMs is their capacity for transfer learning. They can be initially trained on a vast dataset and subsequently adjusted for particular tasks. This diminishes the necessity for extensive task-specific datasets and expedites the advancement of natural language processing applications.'

In [20]:
count_tokens(sequential_chain, {'text': input_text, 'style': 'academic'})

Spent a total of 219 tokens


'Deep learning serves as the foundation for expansive language models, empowering them to comprehend and produce human language. These models have demonstrated exceptional accomplishments in a multitude of language-related endeavors, fundamentally transforming the realm of natural language processing. A significant benefit of these large language models is their capacity for transfer learning, whereby they can be initially trained on a vast dataset and subsequently fine-tuned for specific tasks. This approach minimizes the necessity for extensive task-specific datasets and expedites the advancement of natural language processing applications.'